In [1]:
import itertools
import numpy as np
import pandas as pd
import concurrent.futures
import numpy as np
import math
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, cross_validate, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_curve, ConfusionMatrixDisplay, auc, roc_auc_score, f1_score, confusion_matrix
from sklearn import datasets
from statistics import mean
import includes.model as mod
import pandas as pd
from joblib import dump, load
from includes.config import Config
import includes.model_functions as mf
import time
from itertools import combinations
import random
from graphviz import Digraph
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
from datetime import datetime 
import os
import argparse
from itertools import count
from ucimlrepo import fetch_ucirepo 
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier

# Finding new datasets 

I like the datasets that I have at the moment, but it's helpful having some more ones to compare

In [ ]:
# AN EXAMPLE
df = fetch_ucirepo(id=864) 
df = df.data.original
df.rename({'Room_Occupancy_Count': 'Y'}, axis=1, inplace=True)
# df
df = mf.one_hot_encode(df, "Date")
df = mf.one_hot_encode(df, "Time")
# df.to_csv("data/room_occupancy_data.csv")
df

In [34]:
# Academic dropouts
# 3 classes
df = fetch_ucirepo(id=697) 
df = df.data.original
df.rename({'Target': 'Y'}, axis=1, inplace=True)
tuple(df['Y'].unique())
df.to_csv("data/academic_dropout.csv")
df

,Marital Status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Y
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,1,1,6,9773,1,1,125.0,1,1,1,...,0,6,8,5,12.666667,0,15.5,2.8,-4.06,Graduate
4420,1,1,2,9773,1,1,120.0,105,1,1,...,0,6,6,2,11.000000,0,11.1,0.6,2.02,Dropout
4421,1,1,1,9500,1,1,154.0,1,37,37,...,0,8,9,1,13.500000,0,13.9,-0.3,0.79,Dropout
4422,1,1,1,9147,1,1,180.0,1,37,37,...,0,5,6,5,12.000000,0,9.4,-0.8,-3.12,Graduate


In [36]:
# Wine Quality
# 11 classes
df = fetch_ucirepo(id=186) 
df = df.data.original
df.rename({'quality': 'Y'}, axis=1, inplace=True)
# tuple(df['Y'].unique())
df = mf.one_hot_encode(df, "color")
df.to_csv("data/wine_quality.csv")
df

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,Y,color_red,color_white
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0,1
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0,1
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0,1
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0,1


In [ ]:
# Healthy Aging
# 3 classes
df = fetch_ucirepo(id=936) 
df

In [38]:
# Maternal Health Risk
# 3 classes
df = fetch_ucirepo(id=863) 
df = df.data.original
df.rename({'RiskLevel': 'Y'}, axis=1, inplace=True)
print(tuple(df['Y'].unique()))
df.to_csv("data/maternal_health_risk.csv")

('high risk', 'low risk', 'mid risk')


In [17]:
# Myocardial infarction complications
# 8 classes
df = fetch_ucirepo(id=579) 
df = df.data.original
df.rename({'LET_IS': 'Y'}, axis=1, inplace=True)
tuple(df['Y'].unique())
# df["FIBR_PREDS"].unique()
df.drop(["ID", "FIBR_PREDS",	"PREDS_TAH","JELUD_TAH",	"FIBR_JELUD",	"A_V_BLOK",	"OTEK_LANC",	"RAZRIV",	"DRESSLER",	"ZSN",	"REC_IM",	"P_IM_STEN"], axis=1, inplace=True)
df


,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,IBS_NASL,GB,SIM_GIPERT,DLIT_AG,...,NOT_NA_2_n,NOT_NA_3_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,Y
0,77.0,1,2.0,1.0,1.0,2.0,NaN,3.0,0.0,7.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0
1,55.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0
2,52.0,1,0.0,0.0,0.0,2.0,NaN,2.0,0.0,2.0,...,2.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0
3,68.0,0,0.0,0.0,0.0,2.0,NaN,2.0,0.0,3.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0
4,60.0,1,0.0,0.0,0.0,2.0,NaN,3.0,0.0,7.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,77.0,0,0.0,4.0,2.0,1.0,NaN,2.0,0.0,7.0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1696,70.0,0,0.0,6.0,2.0,1.0,NaN,2.0,0.0,7.0,...,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1697,55.0,1,3.0,6.0,2.0,2.0,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,6
1698,79.0,0,2.0,2.0,2.0,1.0,NaN,2.0,0.0,7.0,...,NaN,NaN,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1


In [29]:
# RT-IoT2022
# 12 classes
df = fetch_ucirepo(id=942) 
df = df.data.original
df.rename({'Attack_type': 'Y'}, axis=1, inplace=True)
df = mf.one_hot_encode(df, "proto")
df = mf.one_hot_encode(df, "service")
df.drop("id", axis=1, inplace=True)
tuple(df['Y'].unique())
df.to_csv("data/rt_iot.csv")

In [25]:
# Land Mines
# 5 classes
df = fetch_ucirepo(id=763) 
df = df.data.original
df.rename({'M': 'Y'}, axis=1, inplace=True)
tuple(df['Y'].unique())
df.to_csv("data/land_mines.csv")

In [ ]:
new_files = [
    "wine_quality.csv",
    "academic_dropout.csv",
    "maternal_health_risk.csv",
    "rt_iot.csv",
    "land_mines.csv"
]